In [1]:
#hide
#default_exp tools.more_runs
from nbdev.showdoc import *
from dsblocks.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# More runs

> Performs more runs on previous experiments, each run having a different random seed.

In [2]:
#export
import sys
import argparse

from hpsearch.config.hpconfig import get_experiment_manager
import hpsearch.config.hp_defaults as dflt

In [3]:
#for tests
import pytest
from hpsearch.examples.complex_dummy_experiment_manager import generate_data

## parse_arguments_and_run

In [4]:
#export
def parse_args (args):
    parser = argparse.ArgumentParser(description='run experiment')
    parser.add_argument('-d', '--debug', action= "store_true")
    parser.add_argument('-e', '--experiments', type=int, nargs='+', default=None,  help="experiment numbers")
    parser.add_argument('--runs', type=int, default=None,  help="number of runs")
    parser.add_argument('--folder', type=str, default=None, help='name of experiments folder')
    parser.add_argument('-p', '--path', default=dflt.manager_path, type=str)
    pars = parser.parse_args(args)
    
    return pars

def parse_arguments_and_run (args):
    
    pars = parse_args(args)

    em_args = dict(use_process=not pars.debug)
    
    em = get_experiment_manager (manager_path=pars.path)
    
    em.rerun_experiment (experiments= pars.experiments, nruns = pars.runs, 
                         **em_args)

def main():
    parse_arguments_and_run (sys.argv[1:])

### usage

In [5]:
#export tests.tools.test_more_runs
def test_parse_arguments_and_run_more_runs ():
    em = generate_data ('test_parse_arguments_and_run_more_runs')

    df = em.get_experiment_data ()
    assert df.shape==(9,25)

    args = ['-e', '4', '3', '-p', em.manager_path]
    parse_arguments_and_run (args)
    em.raise_error_if_run=True
    df = em.get_experiment_data ()
    x=[f'{i}_validation_accuracy' for i in range(5)]; assert df.columns.isin(x).sum()==5
    assert df.shape==(9,25)

    args = ['-e', '4', '3', '--runs', '10', '-p', em.manager_path]
    em.raise_error_if_run=False
    parse_arguments_and_run (args)
    df = em.get_experiment_data ()
    assert df.shape==(9,45)
    x=[f'{i}_validation_accuracy' for i in range(10)]; assert df.columns.isin(x).sum()==10
    
    em.remove_previous_experiments (parent=True)

In [6]:
tst.run (test_parse_arguments_and_run_more_runs, tag='dummy', debug=False)

running test_parse_arguments_and_run_more_runs
